In [1]:
import tweepy,re
from tweepy import Stream
#from tweepy.stream import StreamListener 
from tweepy import OAuthHandler
import json
from geotext import GeoText
from urllib2 import urlopen
from bs4 import BeautifulSoup
from nltk.tag import StanfordNERTagger
import urllib,urllib2
from lxml import html
import requests
import snowballstemmer
from nltk.corpus import stopwords
import ssl
from geopy.geocoders import Nominatim
from gensim.summarization import summarize
from collections import defaultdict
from textblob import TextBlob

In [2]:
def load_api():
    ''' Function that loads the twitter API after authorizing the user. '''

    consumer_key = 'qiOaRLVPldazXmYmF3IaIQw4L'
    consumer_secret = 'FAzcCMF1UUyuNuSeddAA1nDJYPeXm6OhaCD084k1t3BZ0HleCY'
    access_token = '717220472623071233-oeDgXLyYdqT92Mi06aaAGV7EtSExfKS'
    access_secret = 'tVh6WAqVqJ5Pekb3skPON4OD46dyyBAIGOiWjPkZrtglC'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

In [3]:
tweets_data = []
api = load_api()
search_phrases = ['#EdPolicy','#EdReform','#CommonCore','#CCSS','#CCChat','#ESSA']
maxTweets = 100

In [6]:
tweets_data = []
#f = open('tweets.json','a')
for search_phrase in search_phrases:
    for status in tweepy.Cursor(api.search,q=search_phrase,since="2017-11-30",until="2017-12-9",lang="en").items(maxTweets):
        tweets_data.append(status._json)
        #f.write('\n')
    #csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])

In [7]:
print len(tweets_data)

500


In [8]:
for tweet in tweets_data:
    if tweet['entities']['hashtags']:
        print tweet['entities']['hashtags']
    else:
        tweets_data.remove(tweet)

[{u'indices': [99, 108], u'text': u'edpolicy'}, {u'indices': [109, 117], u'text': u'pblchat'}]
[{u'indices': [97, 107], u'text': u'HKFellows'}]
[{u'indices': [88, 95], u'text': u'mathed'}, {u'indices': [96, 103], u'text': u'stemed'}, {u'indices': [104, 119], u'text': u'deeperlearning'}, {u'indices': [120, 129], u'text': u'edpolicy'}, {u'indices': [130, 138], u'text': u'pblchat'}]
[{u'indices': [37, 52], u'text': u'deeperlearning'}, {u'indices': [124, 133], u'text': u'edpolicy'}, {u'indices': [134, 142], u'text': u'pblchat'}]
[{u'indices': [37, 52], u'text': u'deeperlearning'}, {u'indices': [124, 133], u'text': u'edpolicy'}, {u'indices': [134, 142], u'text': u'pblchat'}]
[{u'indices': [99, 108], u'text': u'edpolicy'}, {u'indices': [109, 117], u'text': u'pblchat'}]
[{u'indices': [29, 38], u'text': u'EdPolicy'}]
[{u'indices': [37, 52], u'text': u'deeperlearning'}, {u'indices': [124, 133], u'text': u'edpolicy'}, {u'indices': [134, 142], u'text': u'pblchat'}]
[{u'indices': [101, 110], u'tex

In [9]:
print len(tweets_data)

444


In [10]:
stanford_ner_dir = '/home/dell/stanford-ner-2015-04-20/'
eng_model_filename= stanford_ner_dir + 'classifiers/english.all.3class.distsim.crf.ser.gz'
my_path_to_jar= stanford_ner_dir + 'stanford-ner.jar'

st = StanfordNERTagger(model_filename=eng_model_filename, path_to_jar=my_path_to_jar)
enc = lambda x: x.encode('latin1', errors='ignore')
# regex for url and contact number
#url_re = "(http|ftp|https|www).://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
url_re = "^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$"
tel_re = "^[t][e][l][0-9-:()]+"
#list of all stopwords in english language
stopwords = set(stopwords.words('english'))
stemmer = snowballstemmer.stemmer('english')

In [11]:
def get_urltext(url):
    try:
        context = ssl._create_unverified_context()
        page = urllib2.urlopen(url,context = context).read()
        #r = requests.get(url)
    #except urllib2.HTTPError, e:
    except:
        return None
    soup = BeautifulSoup(page,"html.parser")
    content = "".join([p.get_text() for p in soup.find_all('p')])
    return content

In [12]:
def stopwords_removal(text):
    text = re.sub(r'[^A-Za-z0-9]', ' ', text)
    clean_text = text.split(' ')
    text = []
    for i in range(len(clean_text)):
        if clean_text[i] not in stopwords and len(clean_text[i])>0:
            text.append(clean_text[i])
    return ' '.join(text)

In [13]:
def get_location(text):
    text = text.title()
    places = GeoText(text)
    return places.cities

def most_common(lst):
    return max(set(lst), key=lst.count)

In [14]:
def get_place_name(coord):
    coords = re.split(',|\s+',coord)
    #print coords
    geolocator = Nominatim()
    try:
        location = geolocator.reverse("{}, {}".format(coords[1],coords[2]))
        return get_location(location.address)
    except:
        return None
    
def get_country_name(coord):
    coords = re.split(',|\s+',coord)
    geolocator = Nominatim()
    try:
        location = geolocator.reverse("{}, {}".format(coords[1],coords[2]))
        address = location.address
        address = address.title()
        places = GeoText(address)
        return places.countries[0]
    except:
        return None

In [15]:
def get_hashtag(tweet,search_phrases):
    hashtags = tweet['entities']['hashtags']
    hashtag_list = []
    for d in hashtags:
        hashtag = '#'+d['text']
        if hashtag.lower() in [search_phrase.lower() for search_phrase in search_phrases]:
            hashtag_list.append(hashtag)
    return hashtag_list

In [26]:
for tweet in tweets_data:
    print get_hashtag(tweet,search_phrases)

[u'#edpolicy']
[u'#ESSA']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#Edpolicy']
[u'#Edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[]
[u'#edpolicy']
[]
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#EdPolicy']
[u'#ESSA', u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#ESSA', u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#ESSA', u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#EdPolicy']
[u'#edpolicy']
[u'#edpolicy']
[]
[u'#edpolicy']
[u'#edpolicy']
[u'#ESSA']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy', u'#edreform']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[]
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy']
[u'#edpolicy', u'#edreform']
[u'#EdPolicy']
[u'#edpo

In [14]:
def update_dict(location,text,hashtag):
    global tweets_dict
    if hashtag in tweets_dict.keys():
        d = {'location' : location, 'text': text}
        tweets_dict[hashtag].append(d)
    else:
        tweets_dict[location] = []
        d = {'location' : location, 'text' : text}
        tweets_dict[location].append(d)

In [15]:
count = 0
global tweets_dict 
tweets_dict = defaultdict(list)
for tweet in tweets_data:
    count += 1
    if len(tweet['entities']['urls'])>0:
        text = tweet['text']
        exp_url_text = get_urltext(tweet['entities']['urls'][0]['expanded_url'])
        if exp_url_text is not None and exp_url_text :
            text += exp_url_text
        loc = get_location(text)
        if len(loc)>0:
            location = most_common(loc)
            #print location
            hashtag = get_hashtag(tweet,search_phrases)
            update_dict(location,text,hashtag)
        else:
            location = tweet['user']['location'].lower()
            if not re.search(url_re,location) and not re.search(tel_re,location) :
                if ':' in location:
                    loc = get_place_name(re.split(':',location)[1])
                    if loc is None:
                        loc = get_location(location)
                else :
                    loc = get_location(location)
                if len(loc) > 0:
                    location = loc[0]
                    hashtag = get_hashtag(tweet,search_phrases)
                    update_dict(location,text,hashtag)
                else:
                    #location = tweet['user']['location'].encode('utf-8')
                    if ':' in location:
                        point = re.split(':',location)[1]
                        country = get_country_name(point)
                        if country is not None and country:
                            location = country
                    hashtag = get_hashtag(tweet,search_phrases)
                    update_dict(location,text,hashtag)
    else:
        text = tweet['text']
        loc = get_location(text)
        if len(loc) > 0:
            location = most_common(loc)
            hahstag = get_hashtag(tweet,search_phrases)
            update_dict(location,text,hashtag)
        else:
            location = tweet['user']['location'].lower()
            if not re.search(url_re,location) and not re.search(tel_re,location) :
                if ':' in location:
                    loc = get_place_name(re.split(':',location)[1])
                    if loc is  None:
                        loc = get_location(location)
                else :
                    loc = get_location(location)
                if len(loc) > 0:
                    location = loc[0]
                    hashtag = get_hashtag(tweet,search_phrases)
                    update_dict(location,text,hashtag)
                else:
                    #location = tweet['user']['location'].encode('utf-8')
                    if ':' in location:
                        point = re.split(':',location)[1]
                        country = get_country_name(point)
                        if country is not None and country :
                            location = country
                    hashtag = get_hashtag(tweet,search_phrases)
                    update_dict(location,text,hashtag)

UnboundLocalError: local variable 'hashtag' referenced before assignment

In [ ]:
positive_dicts = defaultdict(list)
negative_dicts = defaultdict(list)
neutral_dicts = defaultdict(list)
for location in tweets_dict.keys():
    for text in tweets_dict[location]:
        #print type(text)
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            if location in positive_dicts.keys():
                positive_dicts[location].append(text)
            else:
                positive_dicts[location] = []
                positive_dicts[location].append(text)
        elif analysis.sentiment.polarity < 0:
            if location in negative_dicts.keys():
                negative_dicts[location].append(text)
            else:
                negative_dicts[location] = []
                negative_dicts[location].append(text)
        else:
            if location in neutral_dicts.keys():
                neutral_dicts[location].append(text)
            else:
                neutral_dicts[location] = []
                neutral_dicts[location].append(text)